<a href="https://colab.research.google.com/github/sudipsiwakoti/Assignment_2/blob/master/Assignment2_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#importing libraries  
!pip install pandas
import pandas as pd
from google.colab import drive


In [14]:
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
!ls "/content/drive/My Drive/Machine learning"

Emission_methane.csv


In [0]:
methane_df = pd.read_csv("/content/drive/My Drive/Machine learning/Emission_methane.csv", skiprows= 4, error_bad_lines=False)


In [25]:
total_methane= methane_df.groupby('Country Name').sum()
methane_df_transpose = total_methane.transpose()
Total_methane= methane_df_transpose.sum(1)
Total_methane= Total_methane.reset_index()
Total_methane = Total_methane.rename(columns={"index":"Year"})
print(Total_methane)

    Year            0
0   1960     0.000000
1   1961  7743.493040
2   1962  7751.492186
3   1963  7788.391770
4   1964  7808.624409
5   1965  7833.469152
6   1966  7848.466808
7   1967  7856.544746
8   1968  7865.416238
9   1969  7889.806077
10  1970  7876.246508
11  1971  7869.222701
12  1972  7872.559665
13  1973  7903.505193
14  1974  7897.178457
15  1975  7883.813641
16  1976  7894.985494
17  1977  7910.899834
18  1978  7925.769069
19  1979  7935.202822
20  1980  7951.947183
21  1981  7961.941319
22  1982  7918.248463
23  1983  7918.026281
24  1984  7961.509703
25  1985  7979.505062
26  1986  8001.798465
27  1987  8024.330567
28  1988  8042.160737
29  1989  8043.161341
30  1990  8034.140825
31  1991  8140.581866
32  1992  9289.606042
33  1993  9488.841703
34  1994  9471.757465
35  1995  9469.241490
36  1996  9478.788143
37  1997  9499.789702
38  1998  9510.989794
39  1999  9533.804491
40  2000  9621.274002
41  2001  9607.316416
42  2002  9593.781855
43  2003  9589.081580
44  2004  